<a href="https://colab.research.google.com/github/Shilpa393/ML-Health-Project/blob/main/Health_Proj_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
admission = pd.read_csv('/content/drive/MyDrive/Healthcare Project/Healthcare Project/ADMISSIONS.csv')
patients = pd.read_csv('/content/drive/MyDrive/Healthcare Project/Healthcare Project/PATIENTS.csv')
chartevents = pd.read_csv('/content/drive/MyDrive/Healthcare Project/Healthcare Project/CHARTEVENTS.csv')
icustays = pd.read_csv('/content/drive/MyDrive/Healthcare Project/Healthcare Project/ICUSTAYS.csv')

<ipython-input-2-5ce2bb08091b>:3: DtypeWarning: Columns (8,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  chartevents = pd.read_csv('/content/drive/MyDrive/Healthcare Project/Healthcare Project/CHARTEVENTS.csv')


In [ ]:
admission.head(2)

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1
1,12263,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,NaN,NaN,HEPATITIS B,1,1


In [ ]:
patients.head(2)

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,9467,10006,F,2094-03-05 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,1
1,9472,10011,F,2090-06-05 00:00:00,2126-08-28 00:00:00,2126-08-28 00:00:00,NaN,1


In [ ]:
chartevents.head(2)

,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped
0,5279021,40124,126179,279554.0,223761,2130-02-04 04:00:00,2130-02-04 04:35:00,19085,95.9,95.9,?F,0.0,0.0,NaN,NaN
1,5279022,40124,126179,279554.0,224695,2130-02-04 04:25:00,2130-02-04 05:55:00,18999,2222221.7,2222221.7,cmH2O,0.0,0.0,NaN,NaN


In [ ]:
icustays.head(2)

,row_id,subject_id,hadm_id,icustay_id,dbsource,first_careunit,last_careunit,first_wardid,last_wardid,intime,outtime,los
0,12742,10006,142345,206504,carevue,MICU,MICU,52,52,2164-10-23 21:10:15,2164-10-25 12:21:07,1.6325
1,12747,10011,105331,232110,carevue,MICU,MICU,15,15,2126-08-14 22:34:00,2126-08-28 18:59:00,13.8507


#Data Preprocessing

###Data preprocessing of dataframes

In [ ]:
patients.head(2)

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,9467,10006,F,2094-03-05 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,1
1,9472,10011,F,2090-06-05 00:00:00,2126-08-28 00:00:00,2126-08-28 00:00:00,NaN,1


In [ ]:
#Creating a copy of the patients dataframe
patients_copy = patients.copy()

In [ ]:
#Dropping off the columns from the patients dataframe
patients_copy = patients_copy.drop(columns = ['row_id','dod_hosp','dod_ssn','expire_flag'])

In [ ]:
patients_copy.head(2)

,subject_id,gender,dob,dod
0,10006,F,2094-03-05 00:00:00,2165-08-12 00:00:00
1,10011,F,2090-06-05 00:00:00,2126-08-28 00:00:00


In [ ]:
#Converting dob and dod to date_time
patients_copy['dob'] = pd.to_datetime(patients_copy['dob'])
patients_copy['dod'] = pd.to_datetime(patients_copy['dod'])

In [ ]:
#One-hot encoding for gender
from sklearn.preprocessing import OneHotEncoder
encode = OneHotEncoder(sparse = False)
gender_encoded = encode.fit_transform(patients[['gender']])
gender_encoded_df = pd.DataFrame(gender_encoded,columns = encode.get_feature_names_out(['gender']))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
#Concatenating the patients dataframe and gender_encoded_df dataframe
patients_copy_1 = pd.concat([patients_copy,gender_encoded_df],axis = 1)

In [ ]:
patients_copy_1.head(2)

,subject_id,gender,dob,dod,gender_F,gender_M
0,10006,F,2094-03-05,2165-08-12,1.0,0.0
1,10011,F,2090-06-05,2126-08-28,1.0,0.0


In [ ]:
#Dropping the original gender column as it is binary encoded
patients_copy_1= patients_copy_1.drop(columns = ['gender'])

In [ ]:
patients_copy_1.head(2)

,subject_id,dob,dod,gender_F,gender_M
0,10006,2094-03-05,2165-08-12,1.0,0.0
1,10011,2090-06-05,2126-08-28,1.0,0.0


In [ ]:
#Type casting of gender encoded columns into integer as it is in float
patients_copy_1[['gender_F','gender_M']] = patients_copy_1[['gender_F','gender_M']].astype(int)

In [ ]:
patients_copy_1.head(2)

,subject_id,dob,dod,gender_F,gender_M
0,10006,2094-03-05,2165-08-12,1,0
1,10011,2090-06-05,2126-08-28,1,0


###Data preprocessing of admissions dataframe

In [ ]:
admission.head(2)

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1
1,12263,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,NaN,NaN,HEPATITIS B,1,1


In [ ]:
# Creating a copy of admission dataframe
admission_copy = admission.copy()

In [ ]:
# Dropping the columns from the admission dataframe
admission_copy = admission_copy.drop(columns = ['row_id','insurance','language','religion','marital_status','ethnicity','edregtime','edouttime'])

In [ ]:
# Converting admittime,dischtime into datetime format
admission_copy['admittime'] = pd.to_datetime(admission_copy['admittime'])
admission_copy['dischtime'] = pd.to_datetime(admission_copy['dischtime'])

In [ ]:
# Replacing NaN values of the deathtime column with Unknown where hospital_expire_flag is 0
admission_copy['deathtime'] = admission_copy['deathtime'].fillna('Unknown')

In [ ]:
# Checking if the NaN values are replaced with 'Unknown' only for hospital_expire_flag = 0
death_time = admission_copy[admission_copy['hospital_expire_flag']==0]
death_time.head(2)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,diagnosis,hospital_expire_flag,has_chartevents_data
0,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,Unknown,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,SEPSIS,0,1
3,10017,199207,2149-05-26 17:19:00,2149-06-03 18:42:00,Unknown,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,HUMERAL FRACTURE,0,1


In [ ]:
#Converting the deathtime dates to datetime format where hospital_expire_flag is 1
admission_copy.loc[admission_copy['hospital_expire_flag']==1,'deathtime'] = pd.to_datetime(admission_copy.loc[admission_copy['hospital_expire_flag']==1,'deathtime'])

In [ ]:
# Checking if the datetime values are handled with proper datetime format only for hospital_expire_flag = 1
death_time = admission_copy[admission_copy['hospital_expire_flag']==1]
death_time.head(2)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,diagnosis,hospital_expire_flag,has_chartevents_data
1,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,HEPATITIS B,1,1
2,10013,165520,2125-10-04 23:36:00,2125-10-07 15:13:00,2125-10-07 15:13:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,SEPSIS,1,1


In [ ]:
#Finding the unique values of admission_type
admission_copy['admission_type'].unique()

array(['EMERGENCY', 'ELECTIVE', 'URGENT'], dtype=object)

In [ ]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.5 MB/s eta 0:00:00


In [ ]:
#Binary encoding for the admission_type column
import category_encoders as ce
encode = ce.BinaryEncoder(cols = ['admission_type'])
admission_copy = encode.fit_transform(admission_copy)

In [ ]:
#Checking unique values for the column - admission_location
admission_copy['admission_location'].unique()

array(['EMERGENCY ROOM ADMIT', 'TRANSFER FROM HOSP/EXTRAM',
       'PHYS REFERRAL/NORMAL DELI', 'CLINIC REFERRAL/PREMATURE',
       'TRANSFER FROM SKILLED NUR'], dtype=object)

In [ ]:
#One-Hot encoding for the admission_location column
from sklearn.preprocessing import OneHotEncoder
encode = OneHotEncoder(sparse = False)
ad_loc_encode = encode.fit_transform(admission_copy[['admission_location']])
ad_loc_encode_df = pd.DataFrame(ad_loc_encode,columns = encode.get_feature_names_out(['admission_location'])).astype(int)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
admission_copy_1 = pd.concat([admission_copy,ad_loc_encode_df],axis = 1)

In [ ]:
admission_copy_1.head(2)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type_0,admission_type_1,admission_location,discharge_location,diagnosis,hospital_expire_flag,has_chartevents_data,admission_location_CLINIC REFERRAL/PREMATURE,admission_location_EMERGENCY ROOM ADMIT,admission_location_PHYS REFERRAL/NORMAL DELI,admission_location_TRANSFER FROM HOSP/EXTRAM,admission_location_TRANSFER FROM SKILLED NUR
0,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,Unknown,0,1,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,SEPSIS,0,1,0,1,0,0,0
1,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,0,1,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,HEPATITIS B,1,1,0,0,0,1,0


In [ ]:
#Dropping off admission_location column as it is encoded
admission_copy_1 = admission_copy_1.drop(columns = ['admission_location'])

In [ ]:
admission_copy_1.head(2)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type_0,admission_type_1,discharge_location,diagnosis,hospital_expire_flag,has_chartevents_data,admission_location_CLINIC REFERRAL/PREMATURE,admission_location_EMERGENCY ROOM ADMIT,admission_location_PHYS REFERRAL/NORMAL DELI,admission_location_TRANSFER FROM HOSP/EXTRAM,admission_location_TRANSFER FROM SKILLED NUR
0,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,Unknown,0,1,HOME HEALTH CARE,SEPSIS,0,1,0,1,0,0,0
1,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,0,1,DEAD/EXPIRED,HEPATITIS B,1,1,0,0,0,1,0


###Data preprocessing for icustays dataframe